# Unit Commitment

TODO

In [1]:
using Distributed
# addprocs(17 - nprocs())
@everywhere using Pkg
@everywhere Pkg.activate(".")

  Activating environment at `~/Packages/MIPLearn/dev/docs/benchmarks/Project.toml`


In [2]:
@everywhere using MIPLearn
@everywhere using MIPLearn.BB
@everywhere using UnitCommitment
@everywhere using Gurobi
@everywhere using Random
@everywhere using JuMP
@everywhere using Logging
@everywhere import UnitCommitment: XavQiuAhm2021
@everywhere Logging.disable_logging(Logging.Warn);

┌ Info: Precompiling MIPLearn [2b1277c3-b477-4c49-a15e-7ba350325c68]
└ @ Base loading.jl:1317


In [ ]:
@everywhere Base.@kwdef struct UnitCommitmentData
    case::String
    hours::Int
    seed::Int
end

@everywhere function build_uc_model(data::UnitCommitmentData)
    instance = UnitCommitment.slice(
        UnitCommitment.read_benchmark("matpower/$(data.case)/2017-02-01"),
        1:data.hours,
    )
    Random.seed!(data.seed)
    UnitCommitment.randomize!(
        instance,
        XavQiuAhm2021.Randomization(),
    )
    model = UnitCommitment.build_model(
        instance = instance,
        variable_names = true,
    )
    @lazycb(
        model,
        UnitCommitment.find_lazy,
        UnitCommitment.enforce_lazy,
    )
    return model
end

instances = MIPLearn.save(
    [
        UnitCommitmentData(
            case = "case1888rte",
            hours = 4,
            seed = i,
        )
        for i in 1:50
    ],
    "uc",
)

MIPLearn.run_benchmarks(
    optimizer = Gurobi.Optimizer,
    train_instances = instances[1:45],
    test_instances = instances[46:50],
    build_model = build_uc_model,
    progress = true,
)

0.0%┣                                               ┫ 0/45 [00:00<00:-2, -0s/it]
2.2%┣█                                          ┫ 1/45 [00:42<Inf:Inf, InfGs/it]
